## Download files

In [1]:
! wget https://github.com/futurexskill/ml-model-deployment/raw/main/tfidfmodel.pickle

--2021-04-20 08:46:10--  https://github.com/futurexskill/ml-model-deployment/raw/main/tfidfmodel.pickle
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/tfidfmodel.pickle [following]
--2021-04-20 08:46:11--  https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/tfidfmodel.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49660 (48K) [application/octet-stream]
Saving to: ‘tfidfmodel.pickle.1’

tfidfmodel.pickle.1 100%[===================>]  48.50K  --.-KB/s    in 0.01s   

2021-04-20 08:46:11 (3.97 MB/s) - ‘tfidfmodel.pickle.1’ saved [49

In [2]:
! wget https://github.com/futurexskill/ml-model-deployment/raw/main/textclassifier.pickle

--2021-04-20 08:46:11--  https://github.com/futurexskill/ml-model-deployment/raw/main/textclassifier.pickle
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/textclassifier.pickle [following]
--2021-04-20 08:46:12--  https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/textclassifier.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3235575 (3.1M) [application/octet-stream]
Saving to: ‘textclassifier.pickle.1’

textclassifier.pick 100%[===================>]   3.08M  19.4MB/s    in 0.2s    

2021-04-20 08:46:13 (19.4 MB/s) - ‘textclassif

In [3]:
!ls

sample_data	       textclassifier.pickle.1	tfidfmodel.pickle.1
textclassifier.pickle  tfidfmodel.pickle


## Connect to twitter and get tweets

In [4]:
import tweepy

In [5]:
consumer_key = '7HZMvRtHwYJdZbHFrL9tEWBMR'
consumer_secret = 'qaxxnBMZc9m3bYarMm27lIa2GpYgbM5T3m8i6rNxIzYOQZUM2n'
access_token = '1382840426-rxV0lhFSdD86Gzy8XjuO9IIkSkfDlofOguEop9d'
access_secret = 'IkNHb227OxkHnN1gm5bdZ4abkThkQvAQADTHgVUi9CtHt'

In [6]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

In [7]:
api = tweepy.API(auth, timeout=20)

In [8]:
tweet_text = 'vaccine'


In [9]:
tweets_list = []

In [10]:
for status in tweepy.Cursor(api.search, q=tweet_text, lang='en', result_type='recent').items(500):
  tweets_list.append(status.text)

In [11]:
len(tweets_list)

500

In [12]:
tweets_list[10]

'RT @muglikar_: When Corona Virus had entered India last year, I had tweeted to Modi that we will need a vaccine.\n\nNot only Modi, but all sc…'

## Clean tweets and do sentiment analysis

In [13]:
import re
import pickle

In [14]:
clean_tweets = tweets_list.copy()
for i in range(len(tweets_list)):

  tweet = tweets_list[i].lower()
  # remove junk charactres
  tweet = re.sub(r'\W', ' ', tweet)
  # remove one or more spaces
  tweet = re.sub(r'\s+', ' ', tweet)
  # remove all chars except a to z and A to Z. Replace with a space
  tweet = re.sub('[^a-zA-Z]', ' ', tweet)
  
  clean_tweets[i] = tweet

In [15]:
clean_tweets[10]

'rt muglikar  when corona virus had entered india last year i had tweeted to modi that we will need a vaccine not only modi but all sc '

In [16]:
with open('textclassifier.pickle', 'rb') as file:
  classifier = pickle.load(file)

In [17]:
with open('tfidfmodel.pickle', 'rb') as file:
  tfidf = pickle.load(file)

In [18]:
positive_tweet = []
negative_tweet = []

In [19]:
i = 0
for tweet in clean_tweets:
  sentiment = classifier.predict(tfidf.transform([tweet]).toarray())
  if sentiment[0] == 1:
    positive_tweet.append(i)
  else:
    negative_tweet.append(i)
  i += 1

In [20]:
len(positive_tweet), len(negative_tweet)

(99, 401)

In [21]:
tweets_list[positive_tweet[10]]

'RT @kylegriffin1: Fauci to CBS on vaccine skeptics: "The ones that keep saying you\'re encroaching on our liberties by asking us to wear mas…'

## Try to get better result with stemming

In [22]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [24]:
import nltk
nltk.download('all')

In [25]:
ps = PorterStemmer()
clean_tweets_1 = tweets_list.copy()
for i in range(len(tweets_list)):

  tweet = tweets_list[i].lower()
  # remove junk charactres
  tweet = re.sub(r'\W', ' ', tweet)
  # remove one or more spaces
  tweet = re.sub(r'\s+', ' ', tweet)
  # remove all chars except a to z and A to Z. Replace with a space
  tweet = re.sub('[^a-zA-Z]', ' ', tweet)
  tweet = re.sub(' +', ' ', tweet)
  tweet = tweet.split()
  tweet = [ps.stem(word) for word in tweet if not word in set(stopwords.words("english"))]
  tweet = " ".join(tweet)
  
  clean_tweets_1[i] = tweet

In [26]:
len(clean_tweets_1)

500

In [27]:
clean_tweets_1[10]

'rt muglikar corona viru enter india last year tweet modi need vaccin modi sc'

In [28]:
clean_tweets[10]

'rt muglikar  when corona virus had entered india last year i had tweeted to modi that we will need a vaccine not only modi but all sc '

In [29]:
positive_tweet_1 = []
negative_tweet_1 = []

i = 0
for tweet in clean_tweets_1:
  sentiment = classifier.predict(tfidf.transform([tweet]).toarray())
  if sentiment[0] == 1:
    positive_tweet_1.append(i)
  else:
    negative_tweet_1.append(i)
  i += 1

In [30]:
len(positive_tweet_1), len(negative_tweet_1)

(110, 390)

In [31]:
for i in (set(positive_tweet_1) - set(positive_tweet)):
  print(f"Positive tweet: {tweets_list[i]}")
  

Positive tweet: @BorisJohnson 

Do not be telling us there are delays to the roadmap. 

The uk has had enough, the vaccine is out t… https://t.co/Vv1NRjYuM6
Positive tweet: RT @GhatakMoloy: India will never forgive PM @narendramodi for underplaying the corona situation in the country and letting so many people…
Positive tweet: RT @FallonTonight: Tonight’s Monologue: @BTS_twt is officially collaborating with McDonald’s, the Biden administration is rolling out vacci…
Positive tweet: @Craig12940777 @trixasis2 Why is every single person getting vaccinated, if the Covid vaccine was approved for EMER… https://t.co/cJLV8blGjw
Positive tweet: RT @FallonTonight: Tonight’s Monologue: @BTS_twt is officially collaborating with McDonald’s, the Biden administration is rolling out vacci…
Positive tweet: RT @JoeBiden: Today, every adult is eligible to get a COVID-19 vaccine. Better days are ahead.
Positive tweet: RT @cosmo_ph: "Jump at the chance to get the vaccine—any brand. Protect yourself and your